In [1]:
#! /usr/bin/env python
# -*- coding:utf-8 -*-
#!/usr/bin/env python
#coding=utf-8

import pandas as pd
import networkx as nx
from copy import deepcopy

In [2]:
#====
EPISODE = 3
NET_EMBED_DIM = 64
#====

In [3]:
cases = pd.read_csv('D:/github_project/Dx2vec-for-Self-harm-prediction/DATA/selfharm_patients_records.csv',na_filter=False)
cases = cases[cases['AGE']!='nnnulll'] # 排除年龄缺失的患者
cases['AGE'] = cases['AGE'].astype(float)
cases = cases[cases['AGE']>10] # 保留 age>10 的患者

print(cases.shape)
cases

(21406, 30)


,OBS,PATID,HOSPITAL,AGE,SEX,DOB,DIST_BOA,ADATE,DDATE,diag_cd_01,...,diag_cd_12,diag_cd_13,diag_cd_14,diag_cd_15,TLOS,DSTATUS,no_code,ayear_int,since2007,selfharm
0,7945753.0,755238.0,b'CMC',69.0,b'M',1937-09-03,b'SHAM SHUI PO',2007-01-01,2007-01-10,496,...,null,null,null,null,9.0,b'X',0.0,2007,0,1
1,7945768.0,1906812.0,b'CMC',32.0,b'M',1974-10-10,b'SHAM SHUI PO',2007-01-01,2007-01-03,305.90,...,null,null,null,null,2.0,b'X',0.0,2007,0,1
2,7946088.0,2326296.0,b'NDH',76.0,b'F',1931-01-01,b'TAI PO',2007-01-01,2007-01-06,434.91,...,null,null,null,null,5.0,b'X',0.0,2007,0,1
3,7946134.0,8290831.0,b'NDH',50.0,b'M',1956-02-13,b'OTHERS',2007-01-01,2007-01-01,305.00,...,null,null,null,null,1.0,b'X',0.0,2007,0,1
4,7946213.0,1160569.0,b'PMH',32.0,b'M',1974-11-19,b'KWAI TSING',2007-01-01,2007-01-01,854.01,...,null,null,null,null,1.0,b'X',0.0,2007,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21530,1418637.0,2127061.0,b'CPH',75.0,b'F',1935-01-01,b'YUEN LONG',2010-12-31,2011-01-02,296.22,...,null,null,null,null,2.0,b'X',0.0,2010,1466,1
21531,1419000.0,6113130.0,b'KWH',66.0,b'M',1944-01-01,b'OTHERS',2010-12-31,2011-01-03,305.90,...,null,null,null,null,3.0,b'X',0.0,2010,1466,1
21532,1419629.0,2127061.0,b'POH',75.0,b'F',1935-01-01,b'YUEN LONG',2010-12-31,2010-12-31,959.0,...,null,null,null,null,1.0,b'X',0.0,2010,1466,1
21533,1419669.0,4319429.0,b'POH',21.0,b'F',1989-01-01,b'OTHERS',2010-12-31,2011-01-02,305.90,...,null,null,null,null,2.0,b'X',0.0,2010,1466,1


In [4]:
# evidence of historical episodes equal or below 3
from collections import Counter # 用于计数可迭代对象中元素的出现次数
allpatid = Counter(list(cases['PATID']))
allpatid = list(allpatid.values())
allpid_above_four = [i for i in allpatid if i>4] # 创建了一个列表，其中包含出现次数大于4的病人的次数
print (len(allpid_above_four)/len(allpatid))

0.10349432450478523


In [5]:
def whether_a_case(df): ##this df has sf record deleted
    if df.iloc[0]['timelength'] <= 365:
        if diags.shape[0]<=6 : return df, True # 如果时间长度满足条件并且诊断的数量不超过 6，则返回整个 DataFrame
        if diags.shape[0]>6  : return df[:6], True # 如果时间长度满足条件但是诊断的数量超过 6，则返回 DataFrame 的前 6 行
    else: return False, False    

患者诊断列表构建

- 获取唯一的患者 id 列表
- 遍历每个患者 id，对其记录进行排序并计算时间长度
- 根据诊断代码和时间长度，使用 whether_a_case 函数筛选符合条件的诊断记录
- 将筛选后的诊断记录添加到 cases_diaglist 列表中，并记录相应的年龄、性别和患者 id（后面代码没用这 3 个 list）

In [7]:
individuals = list(set(list(cases['PATID']))) # 提取不重复 patient id， set() 提取不重复的值
cases_diaglist, sexlist, agelist, pidlist = [], [], [], []
for i in individuals[:]:
    records = cases[cases['PATID'] == i]
    if records.shape[0] >= 2: # 该病人的 record 数量至少为2
        records = records.sort_values(axis = 0, ascending = False, by = 'since2007')
        records.index = range(records.shape[0])
        timelength = []
        for p in records.index[1:2]:
            timelength = records.loc[0]['since2007'] - records.loc[p]['since2007'] # 计算第一个 record 和第二个 record 的时间间隔
        records['timelength'] = timelength
        records = records.drop(0) ##drop first line, i.e., sf record
        
        if records.iloc[0]['diag_cd_01'] != 'null' : # 保留有第一诊断的 patient
            diags = pd.DataFrame(columns=('diags', 'timelength'))
            for j in records.index:
                for m in range(1, 7):
                    if records.loc[j]['diag_cd_0'+str(m)] != 'null':
                        diags.loc[diags.shape[0]] = [records.loc[j]['diag_cd_0'+str(m)], records.loc[j]['timelength']]
            a,b = whether_a_case(diags)
            if (b and (list(a['diags']) not in cases_diaglist)): 
                cases_diaglist.append(list(a['diags']))
                agelist.append(records.iloc[0]['AGE'])
                sexlist.append(records.iloc[0]['SEX'])
                pidlist.append(records.iloc[0]['PATID'])           
len(list(set(pidlist)))  #2314

print(type(cases_diaglist))
cases_diaglist

<class 'list'>


[['965.4', 'E950.0', '571.8'],
 ['997', 'E879', '997.62', 'E878.5', '897', 'E878'],
 ['977.8', 'E950.4', '309.9', '969.0', 'E950.3', '311'],
 ['V71.09'],
 ['486'],
 ['967.9', 'E950.2'],
 ['682.5', '577.1', '571.2', 'V02.62', '577.1', '532.70'],
 ['640.00'],
 ['292.89', '309.9', 'E950.5', '301.7', '305.90', '305.90'],
 ['345.90', 'V02.61', '414.9', '345.90', '287.5', '571.3'],
 ['V61.7', '965.4', 'E935.4'],
 ['296.23', '967.9', 'E950.2', '967.9', 'E950.2', 'E952.1'],
 ['976.0', 'E950.4', '789.06'],
 ['332.0', '977.8', 'E947.8', '296.23', '332.0', '296.22'],
 ['374.9', '682.1'],
 ['305.70', 'E950.4', '728.89', '454.9'],
 ['303.00', '512.0', '958.3', '512.0', '786.50', '569.89'],
 ['527.2', '292.9'],
 ['305.90', 'E950.5', '309.9'],
 ['535.01', '535.50'],
 ['414.9', '424.0', '424.2', '428.9', '429.5', '413.9'],
 ['496'],
 ['313.9', '292.9', 'E950.3', '313.9', '309.0', '309.9'],
 ['296.31', '967.9', 'E950.2', '296.31', '308.0', '296.34'],
 ['854.01', 'E885'],
 ['305.30'],
 ['780.2', '317', 

In [ ]:
f = open('age_cases','w') # 创建文本，写入 agelist
for i in agelist:
    f.write(str(i)+'\n')
f.close()
f = open('sex_cases','w') # 创建文本，写入 sexlist
for i in sexlist:
    if i=="b'F'":
        f.write('0'+'\n')
    else:
        f.write('1'+'\n')
f.close()

In [ ]:
#========
all_case = []
individuals =list(set(list(pidlist)))
for i in individuals[:]:
    ep = dict() # 创建字典，存储历史诊断
    for k in range(EPISODE): ep[k] = [] # e.g., EPISODE == 3, {0: [], 1: [], 2: []}

    records = cases[cases['PATID'] == i]
    records = records.sort_values(axis = 0, ascending = False, by = 'since2007')
    records.index = range(records.shape[0])
    records = records.drop(0) ##drop first line, i.e., sf record
    timeline = set(list(records['since2007']))
    
    count1 = 0 #count the # of episode
    for t in timeline:
        diags_in_an_episode = records[records['since2007'] == t]
        diags_in_an_episode = diags_in_an_episode[['diag_cd_01','diag_cd_02','diag_cd_03','diag_cd_04','diag_cd_05','diag_cd_06']].values
        diags_in_an_episode = list(diags_in_an_episode[0,:]) # 将 NumPy 数组转换为列表
        diags_in_an_episode = list(filter(lambda x : x != 'null', diags_in_an_episode)) # 使用 filter 函数删除列表中的 'null' 值，生成只包含实际诊断的列表
        
        count2 = 0 #count the # of diagnoses in an episode
        for j in diags_in_an_episode:
            ep[count1].append(j)
            if count2>=2: break # 只需要每个 episode 的前3个诊断（0-2）
            count2=count2+1    
        if count1>=2: break # 只需要前3个 episode 的诊断（0-2）
        count1 = count1+1
    all_case.append(ep)

all_case
print(len(all_case))

In [ ]:
#==FIRST THREE DIGIT
# 提取每个 episode 的诊断信息，保留每个诊断的前3个数
temp2= []
for i in all_case: # 每个 patient
    for j in i.keys(): # 每个 episode
        for d in i[j]: # 每个诊断
            l = d.strip().split('.') # 将当前诊断信息按照 . 拆分，并使用 strip 函数去除可能存在的空格
            temp2.append(l[0]) # 将拆分后的诊断信息的第一个部分（前3个数）添加到
        i[j] = temp2 # 用处理后的诊断信息替换原始的诊断信息
        temp2=[]

all_case

In [ ]:
#==nodes in network are not coded in icd, ]
#==so build a dict to traslate    
f = open('D:/github_project/Dx2vec-for-Self-harm-prediction/DATA/emb_selfharm.csv') # 每一行表示节点 + 向量（以非icd编码记录的共病网络）
line = f.readline()
node2vec = dict() # 用于存储节点到向量的映射
nodes_in_net = [] # 用于存储网络中的节点？
while True:
    line = f.readline() # 循环读取每一行
    if not line: break
    l = line.strip().split(' ') # 去除行尾的换行符，并以空格分割字符串
    node2vec[l[0]] = l[1:] # 将分割后的第一个元素作为键，其余部分作为值，存入字典
    nodes_in_net.append(l[0]) # 将分割后的第一个元素添加到节点列表中
f.close() 

node2vec
print(len(node2vec)) # 938个节点共病网络

In [ ]:
#==for nodes not in the network, assign a zero vector 
for i in range(1200):
    if str(i) not in node2vec.keys(): # 检查字符串形式的i是否不在node2vec的键中
        node2vec[str(i)] = [0]*NET_EMBED_DIM # 如果不在，则为该节点赋予一个长度为NET_EMBED_DIM的零向量
        
node2vec
print(len(node2vec))

In [ ]:
#==
icd_netnode = dict()
f = open('D:/github_project/Dx2vec-for-Self-harm-prediction/DATA/icd_category.csv','r')
line = f.readline()
while True:
    line = f.readline()
    if not line:break
    l = line.strip().split(',')
    icd_netnode[l[0]] = l[1]
    
icd_netnode

In [ ]:
#==vectors[patient_indx][episode] = a 64*3 matrix
#all_case[patient_indx][episode][main/second/third_diagnosis]
m,n = len(list(set(pidlist))),3
vectors = [[0 for i in range(n)] for j in range(m)]

for i in range(len(all_case)): # i 表示 patient
    for j in range(EPISODE): # j 表示 episode
        if len(all_case[i][j])==3:
            vectors[i][j] = (node2vec[icd_netnode[all_case[i][j][0]]],
                         node2vec[icd_netnode[all_case[i][j][1]]],
                         node2vec[icd_netnode[all_case[i][j][2]]])
        if len(all_case[i][j])==2:
            vectors[i][j] = (node2vec[icd_netnode[all_case[i][j][0]]],
                         node2vec[icd_netnode[all_case[i][j][1]]],[0]*NET_EMBED_DIM)
        if len(all_case[i][j])==1:
            vectors[i][j] = (node2vec[icd_netnode[all_case[i][j][0]]],[0]*NET_EMBED_DIM,[0]*NET_EMBED_DIM)
        if len(all_case[i][j])==0:
            vectors[i][j] = ([0]*NET_EMBED_DIM, [0]*NET_EMBED_DIM, [0]*NET_EMBED_DIM)
            
vectors

In [ ]:
#===transfer vectors[patient_indx][episode] = a 64*3 matrix
#===to vectors[patient_indx][episode] = a lsit, where 
#===the elements in the list is the absmax            
newvec = []        
for patient_index in range(len(vectors)):
    for episode in range(EPISODE):
        for x, y, z in zip(vectors[patient_index][episode][0], vectors[patient_index][episode][1], vectors[patient_index][episode][2]):
            absmax = max( float(x), float(y), float(z), key=abs )
            newvec.append(absmax)
        vectors[patient_index][episode] = newvec
        newvec = []

f = open('controls.csv','w')
for patient_index in range(len(vectors)):
    for episode in range(EPISODE):
        f.write(str(patient_index)+','+str(episode)+','+str(vectors[patient_index][episode])[1:-1]+'\n')
f.close()

In [ ]:
#============CONTROLS===========
#===============================
#===============================
controls = pd.read_csv('D:/github_project/Dx2vec-for-Self-harm-prediction/DATA/lowrisk_patients_records.csv',na_filter=False)
controls = controls[controls['AGE']!='nnnulll']
controls['AGE'] = controls['AGE'].astype(float)
controls = controls[controls['AGE']>10]
print (controls.shape[0])

In [ ]:
d1 = controls[controls['diag_cd_01']!='null']
d2 = controls[controls['diag_cd_02']!='null']
d3 = controls[controls['diag_cd_03']!='null']
d4 = controls[controls['diag_cd_04']!='null']
d5 = controls[controls['diag_cd_05']!='null']
d6 = controls[controls['diag_cd_06']!='null']
d7 = controls[controls['diag_cd_07']!='null']
d8 = controls[controls['diag_cd_08']!='null']
d9 = controls[controls['diag_cd_09']!='null']
d10 = controls[controls['diag_cd_10']!='null']
(d1.shape[0]+d2.shape[0]+d3.shape[0]+d4.shape[0]+d5.shape[0]+d6.shape[0]+d7.shape[0]+d8.shape[0]+d9.shape[0]+d10.shape[0])/122114

In [ ]:
individuals = list(set(list(controls['PATID'])))
controls_diaglist, sexlist, agelist, pidlist = [], [], [], []
for i in individuals[:]:
    records = controls[controls['PATID'] == i]
    if records.shape[0] >= 1: ##Patients without historical diagnoses will be removed
        agelist.append(records.iloc[0]['AGE'])
        sexlist.append(records.iloc[0]['SEX'])
        pidlist.append(records.iloc[0]['PATID'])           
len(list(set(pidlist)))  # 46460 patients, 136039 records        
f=open('age_controls','w')
for i in agelist:
    f.write(str(i)+'\n')
f.close()
f=open('sex_controls','w')
for i in sexlist:
    if i=="b'F'":
        f.write('0'+'\n')
    else:
        f.write('1'+'\n')
f.close()

In [ ]:
#========
all_control = []
individuals =list(set(list( pidlist)))
for i in individuals[:]:
    ep = dict()
    for k in range(EPISODE): ep[k] = []

    records = controls[controls['PATID'] == i]
    records = records.sort_values(axis = 0, ascending = False, by = 'since2007')
    records.index = range(records.shape[0])
    records = records.drop(0) ##drop first line, i.e., sf record
    timeline = set(list(records['since2007']))
    
    count1 = 0 #count the # of episode
    for t in timeline:
        diags_in_an_episode = records[records['since2007'] == t]
        diags_in_an_episode = diags_in_an_episode[['diag_cd_01','diag_cd_02','diag_cd_03','diag_cd_04','diag_cd_05','diag_cd_06']].values
        diags_in_an_episode = list(diags_in_an_episode[0,:])
        diags_in_an_episode = list(filter(lambda x : x != 'null', diags_in_an_episode))  
        
        count2=0 #count the # of diagnoses in an episode
        for j in diags_in_an_episode:
            ep[count1].append(j)
            if count2>=2: break
            count2=count2+1    
        if count1>=2: break
        count1 = count1+1
    all_control.append(ep)

In [ ]:
#==FIRST THREE DIGIT
temp2= []
for i in all_control:
    for j in i.keys():
        for d in i[j]:
            l = d.strip().split('.')
            temp2.append(l[0])
        i[j] = temp2
        temp2=[]

In [ ]:
#==for nodes not in the network, assign a zero vector 
for i in range(1500):
    if str(i) not in node2vec.keys():
        node2vec[str(i)] = [0]*NET_EMBED_DIM

In [ ]:
#==vectors[patient_indx][episode] = a 64*3 matrix
#all_case[patient_indx][episode][main/second/third_diagnosis]
m,n = len(list(set(pidlist))),3
vectors = [[0 for i in range(n)] for j in range(m)]

for i in range(len(all_control)):
    for j in range(EPISODE):
        if len(all_control[i][j])==3:
            vectors[i][j] = (node2vec[icd_netnode[all_control[i][j][0]]],
                         node2vec[icd_netnode[all_control[i][j][1]]],
                         node2vec[icd_netnode[all_control[i][j][2]]])
        if len(all_control[i][j])==2:
            vectors[i][j] = (node2vec[icd_netnode[all_control[i][j][0]]],
                         node2vec[icd_netnode[all_control[i][j][1]]],[0]*NET_EMBED_DIM)
        if len(all_control[i][j])==1:
            vectors[i][j] = (node2vec[icd_netnode[all_control[i][j][0]]],[0]*NET_EMBED_DIM,[0]*NET_EMBED_DIM)
        if len(all_control[i][j])==0:
            vectors[i][j] = ([0]*NET_EMBED_DIM, [0]*NET_EMBED_DIM, [0]*NET_EMBED_DIM)

In [ ]:
#===transfer vectors[patient_indx][episode] = a 64*3 matrix
#===to vectors[patient_indx][episode] = a lsit, where 
#===the elements in the list is the absmax            
newvec=[]        
for patient_index in range(len(vectors)):
    for episode in range(EPISODE):
        for x, y, z in zip(vectors[patient_index][episode][0], vectors[patient_index][episode][1], vectors[patient_index][episode][2]):
            absmax = max( float(x), float(y), float(z), key=abs )
            newvec.append(absmax)
        vectors[patient_index][episode]=newvec
        newvec=[]

f = open('controls.csv','w')
for patient_index in range(len(vectors)):
    for episode in range(EPISODE):
        f.write(str(patient_index)+','+str(episode)+','+str(vectors[patient_index][episode])[1:-1]+'\n')
f.close()
               

In [ ]:
#==========multi-hot==
diag_set = []
patient_diag = dict()
for i in range(len(all_case)):
    patient_diag[i] = []
    for j in range(EPISODE):
        if len(all_case[i][j])==3:
            patient_diag[i].append(all_case[i][j][0])
            patient_diag[i].append(all_case[i][j][1])
            patient_diag[i].append(all_case[i][j][2])
            diag_set.append(all_case[i][j][0])
            diag_set.append(all_case[i][j][1])
            diag_set.append(all_case[i][j][2])
        if len(all_case[i][j])==2:
            patient_diag[i].append(all_case[i][j][0])
            patient_diag[i].append(all_case[i][j][1])
            diag_set.append(all_case[i][j][0])
            diag_set.append(all_case[i][j][1])
        if len(all_case[i][j])==1:
            patient_diag[i].append(all_case[i][j][0])
            diag_set.append(all_case[i][j][0])

diag_set

In [ ]:
lowrisk_diag = dict()
for i in range(len(all_control)):
    lowrisk_diag[i] = []
    for j in range(EPISODE):
        if len(all_control[i][j])==3:
            lowrisk_diag[i].append(all_control[i][j][0])
            lowrisk_diag[i].append(all_control[i][j][1])
            lowrisk_diag[i].append(all_control[i][j][2])
            diag_set.append(all_control[i][j][0])
            diag_set.append(all_control[i][j][1])
            diag_set.append(all_control[i][j][2])
        if len(all_control[i][j])==2:
            lowrisk_diag[i].append(all_control[i][j][0])
            lowrisk_diag[i].append(all_control[i][j][1])
            diag_set.append(all_control[i][j][0])
            diag_set.append(all_control[i][j][1])
        if len(all_control[i][j])==1:
            lowrisk_diag[i].append(all_control[i][j][0])
            diag_set.append(all_control[i][j][0])
diag_set = list(set(diag_set))

In [ ]:
multihot = [0 for k in range( len(diag_set))]
icd_indx = dict()
for i in range(len(diag_set)):
    icd_indx[diag_set[i]] = i 
#==multihot_cases
f = open('multihot_cases.csv','w')
for i in patient_diag.keys():
    if len(patient_diag[i])>0:
        for j in patient_diag[i]:
            multihot[icd_indx[j]] = multihot[icd_indx[j]]+1
    else: 
        multihot = multihot
    f.write(str(i)+','+str(multihot)[1:-1]+'\n')
    multihot = [0 for k in range( len(diag_set))]
f.close()
#==multihot_controls
f = open('multihot_controls.csv','w')
for i in lowrisk_diag.keys():
    if len(lowrisk_diag[i])>0:
        for j in lowrisk_diag[i]:
            multihot[icd_indx[j]] = multihot[icd_indx[j]]+1
    else: 
        multihot = multihot
    f.write(str(i)+','+str(multihot)[1:-1]+'\n')
    multihot = [0 for k in range( len(diag_set))]
f.close()


In [ ]:
##======sigle diag
# emb
single_diag = pd.DataFrame()
single_diag['diag'] = diag_set
emb =[]
for i in diag_set:
    emb.append(node2vec[icd_netnode[i]])
    
for i in range(NET_EMBED_DIM):
    single_diag['emb_'+str(i)] = 0
single_diag.iloc[:,1:]=emb
for i in range(NET_EMBED_DIM, NET_EMBED_DIM*EPISODE):
    single_diag['emb_'+str(i)] = 0


In [ ]:
# multihot_singles
multihot = [0 for k in range( len(diag_set))]
mhot = []
for i in diag_set:
    multihot[icd_indx[i]] = multihot[icd_indx[i]]+1
    mhot.append(multihot)
    multihot = [0 for k in range( len(diag_set))]
for i in range(len(diag_set)):
    single_diag['mhot_'+str(i)] = 0
single_diag.iloc[:,1+NET_EMBED_DIM*EPISODE:] = mhot
single_diag['AGE'] = 50
single_diag['SEX'] = 0
single_diag['y'] = 2